<a href="https://colab.research.google.com/github/arpit0891/Python-Algorithms/blob/master/Assignment_DSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install SimPy

In [0]:
'''
1.	WAP to simulate car wash center with capacity of two cars in two slots.
2.	WAP to illustrate traffic signal simulation.
3.	WAP to illustrate Priority Resource scheduling.
4.	WAP to provide interrupts in case of traffic light simulation.
5.	WAP to simulate car charging station with two queue each having capacity of 4 slots.
6.	WAP for Process Interaction in any simulation scenario.
7.	WAP to Interrupting another process in case of car charging simulation.
8.	What is the different between dependent and independent processes?'''


'\n1.\tWAP to simulate car wash center with capacity of two cars in two slots.\n2.\tWAP to illustrate traffic signal simulation.\n3.\tWAP to illustrate Priority Resource scheduling.\n4.\tWAP to provide interrupts in case of traffic light simulation.\n5.\tWAP to simulate car charging station with two queue each having capacity of 4 slots.\n6.\tWAP for Process Interaction in any simulation scenario.\n7.\tWAP to Interrupting another process in case of car charging simulation.\n8.\tWhat is the different between dependent and independent processes?'

# 1.	WAP to simulate car wash center with capacity of two cars in two slots.

In [0]:
import random
import simpy
RANDOM_SEED = 20
NEW_Cars = 5  # Total number of cars
INTERVAL_Cars = 3.0  # Generate new cars roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 20  # Max. customer patience
def source(env, number, interval, counter):
    for i in range(number):
        c = car(env, 'Car %02d' % i, counter, time_in_survey=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)
def car(env, name, counter, time_in_survey):
    arrive = env.now
    print('%.2f min %s: Car is ready to take wash' % (arrive, name))
    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        results = yield req | env.timeout(patience)
        wait = env.now - arrive
        if req in results:
            print('%.2f min %s: Waited %.2f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_survey)
            yield env.timeout(tib)
            print('%.2f min %s: Finished after waiting %.2f min' % (env.now, name,wait))
        else:
            print('%.2f min %s: Finished after waiting %.2f min' % (env.now, name, wait))
print('CAR WASH')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = simpy.Resource(env, capacity=2)
env.process(source(env, NEW_Cars, INTERVAL_Cars, counter))
env.run()

CAR WASH
0.00 min Car 00: Car is ready to take wash
0.00 min Car 00: Waited 0.00
7.08 min Car 01: Car is ready to take wash
7.08 min Car 01: Waited 0.00
14.13 min Car 02: Car is ready to take wash
17.46 min Car 00: Finished after waiting 0.00 min
17.46 min Car 02: Waited 3.32
19.20 min Car 01: Finished after waiting 0.00 min
21.19 min Car 03: Car is ready to take wash
21.19 min Car 03: Waited 0.00
21.75 min Car 04: Car is ready to take wash
27.67 min Car 02: Finished after waiting 3.32 min
27.67 min Car 04: Waited 5.92
63.62 min Car 04: Finished after waiting 5.92 min
82.26 min Car 03: Finished after waiting 0.00 min


# 2.WAP to illustrate traffic signal simulation.

In [0]:
from collections import deque
from numpy import random
import simpy
from simpy.util import start_delayed


class Struct(object):
   def __init__(self, **kwargs):
      self.__dict__.update(kwargs)
random.seed([1, 2, 3])
#specify the initial data
end_time= 500.0
arrival_rate= 0.05
t_interarrival_mean= 2.0 / arrival_rate
t_green= 30.0
t_red= 90.0
t_depart_left= 1.6; t_depart_mode= 2.0; t_depart_right= 2.4
queue= deque()
arrival_count= departure_count= 0
Q_stats= Struct(count=0, cars_waiting=0)
W_stats= Struct(count=0, waiting_time=0.0)
def arrival():
   global arrival_count, env, light, queue
   while True:
      arrival_count+= 1
      if light == 'red' or len(queue):
         queue.append((arrival_count, env.now))
         min=env.now/60
         print("Car #%d arrived and joined the queue at position %d at time ""%.1f""sec or ""%.1f""mins"%(arrival_count, len(queue), env.now,min))
      else:
         min=env.now/60
         print("Car #%d arrived to a green light with no cars waiting at time ""%.1f""sec or ""%.1f""mins"%(arrival_count, env.now,min))
         W_stats.count+= 1
         #next arival
      yield env.timeout( random.exponential(t_interarrival_mean))
def departure():
     global env, queue
     while True:
      car_number, t_arrival= queue.popleft()
      min=env.now/60
      print("Car #%d departed at time %.1f""sec or"" %.1f""mins"", leaving %d cars in the queue."%(car_number, env.now,min, len(queue)))
      W_stats.count+= 1
      W_stats.waiting_time+= env.now - t_arrival
      if light == 'red' or len(queue) == 0:
         return
      delay= random.triangular(left=t_depart_left, mode=t_depart_mode,right=t_depart_right)
      yield env.timeout(delay)
def light():
   global env, light
   while True:
      light= 'green'
      min=env.now/60
      print("\nThe light turned green at time %.1f.""sec or ""%.1f""mins"%(env.now,min))
      if len(queue):
         delay= random.triangular(left=t_depart_left, mode=t_depart_mode,right=t_depart_right)
         start_delayed(env, departure(), delay=delay)
      yield env.timeout(t_green)
      light= 'red'
      min=env.now/60
      print("\nThe light turned red at time %.1f.""sec or ""%.1f""mins"% (env.now,min))
      yield env.timeout(t_red)
env= simpy.Environment()
env.process(light())
t_first_arrival= random.exponential(t_interarrival_mean)
start_delayed(env, arrival(), delay=t_first_arrival)
env.run(until=end_time)


The light turned green at time 0.0.sec or 0.0mins

The light turned red at time 30.0.sec or 0.5mins
Car #1 arrived and joined the queue at position 1 at time 37.7sec or 0.6mins

The light turned green at time 120.0.sec or 2.0mins
Car #1 departed at time 122.0sec or 2.0mins, leaving 0 cars in the queue.
Car #2 arrived to a green light with no cars waiting at time 124.8sec or 2.1mins

The light turned red at time 150.0.sec or 2.5mins
Car #3 arrived and joined the queue at position 1 at time 215.9sec or 3.6mins
Car #4 arrived and joined the queue at position 2 at time 217.1sec or 3.6mins

The light turned green at time 240.0.sec or 4.0mins
Car #3 departed at time 242.0sec or 4.0mins, leaving 1 cars in the queue.
Car #4 departed at time 244.3sec or 4.1mins, leaving 0 cars in the queue.

The light turned red at time 270.0.sec or 4.5mins
Car #5 arrived and joined the queue at position 1 at time 314.9sec or 5.2mins
Car #6 arrived and joined the queue at position 2 at time 350.1sec or 5.8mins

3.	WAP to illustrate Priority Resource scheduling.

In [0]:
import simpy
def resource_user(name, env, resource, wait, prio):
     yield env.timeout(wait)
     with resource.request(priority=prio) as req:
         print(f'{name} requesting at {env.now} with priority={prio}')
         yield req
         print(f'{name} got resource at {env.now}')
         yield env.timeout(3)

env = simpy.Environment()
res = simpy.PriorityResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, wait=0, prio=0))
p2 = env.process(resource_user(2, env, res, wait=1, prio=0))
p3 = env.process(resource_user(3, env, res, wait=2, prio=-1))
env.run()

1 requesting at 0 with priority=0
1 got resource at 0
2 requesting at 1 with priority=0
3 requesting at 2 with priority=-1
3 got resource at 3
2 got resource at 6


4.	WAP to provide interrupts in case of traffic light simulation.

In [0]:
class TrafficLight(object):
  def __init__(self,env):
    self.env=env
    self.action=env.process(self.run(env))
  def run(self,env):
    while True:
      print("Red light in ",  env.now)
      red_light_duration=90
      try:
        yield self.env.timeout(red_light_duration)
      except(simpy.Interrupt):
        print("Emergency vehicle arriving")
      print("Yellow light in ",  env.now)
      yellow_light_duration=10
      try:
        yield self.env.timeout(yellow_light_duration)
      except(simpy.Interrupt):
        print("Emergency vehicle arriving")
      print("Green light in ",  env.now)
      green_light_duration=20
      try:
        yield self.env.timeout(green_light_duration)
      except(simpy.Interrupt):
         print("Emergency vehicle arriving")
def emergency(env,tl,t):
  yield env.timeout(t)
  tl.action.interrupt()
env=simpy.Environment()
tl=TrafficLight(env)
env.process(emergency(env,tl,180))
env.run(400)

Red light in  0
Yellow light in  90
Green light in  100
Red light in  120
Emergency vehicle arriving
Yellow light in  180
Green light in  190
Red light in  210
Yellow light in  300
Green light in  310
Red light in  330


# 5.	WAP to simulate car charging station with two queue each having capacity of 4 slots.

In [0]:
import random
import simpy
RANDOM_SEED = 20
NEW_Cars = 10  # Total number of cars
INTERVAL_Cars = 3.0  # Generate new cars roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 20  # Max. customer patience
def source(env, number, interval, counter):
    for i in range(number):
        c = car(env, 'Car %02d' % i, counter, time_in_survey=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)
def car(env, name, counter, time_in_survey):
    arrive = env.now
    print('%.1f min %s: Car is ready to charge' % (arrive, name))
    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        results = yield req | env.timeout(patience)
        wait = env.now - arrive
        if req in results:
            print('%.1f min %s: Waited %.2f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_survey)
            yield env.timeout(tib)
            print('%.1f min %s: Charged after waiting %.2f min' % (env.now, name,wait))
        else:
            print('%.1f min %s: Charged after waiting %.2f min' % (env.now, name, wait))
print('Car Charging Station')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = simpy.Resource(env, capacity=4)
env.process(source(env, NEW_Cars, INTERVAL_Cars, counter))
env.run()

Car Charging Station
0.0 min Car 00: Car is ready to charge
0.0 min Car 00: Waited 0.00
7.1 min Car 01: Car is ready to charge
7.1 min Car 01: Waited 0.00
14.1 min Car 02: Car is ready to charge
14.1 min Car 02: Waited 0.00
17.5 min Car 00: Charged after waiting 0.00 min
19.2 min Car 01: Charged after waiting 0.00 min
21.2 min Car 03: Car is ready to charge
21.2 min Car 03: Waited 0.00
21.7 min Car 04: Car is ready to charge
21.7 min Car 04: Waited 0.00
22.1 min Car 05: Car is ready to charge
22.1 min Car 05: Waited 0.00
23.9 min Car 06: Car is ready to charge
24.3 min Car 02: Charged after waiting 0.00 min
24.3 min Car 06: Waited 0.48
26.6 min Car 05: Charged after waiting 0.00 min
31.0 min Car 06: Charged after waiting 0.48 min
31.0 min Car 07: Car is ready to charge
31.0 min Car 07: Waited 0.00
34.0 min Car 08: Car is ready to charge
34.0 min Car 08: Waited 0.00
34.3 min Car 09: Car is ready to charge
34.4 min Car 08: Charged after waiting 0.00 min
34.4 min Car 09: Waited 0.17
39.2 

# 6.  WAP for Process Interaction in any simulation scenario.
Ans. There are three Process intraction:
*   Sleep until woken up (passivate/reactivate)

*   Waiting for another process to terminate

*  Interrupting another process



**Sleep until woken up**

In [0]:
from random import seed, randint
seed(23)
import simpy
class EV:
    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive(env))
        self.bat_ctrl_proc = env.process(self.bat_ctrl(env))
        self.bat_ctrl_reactivate = env.event()
    def drive(self, env):
        while True:
            # Drive for 20-40 min
            yield env.timeout(randint(20, 40))
            # Park for 1–6 hours
            print('Start parking at', env.now)
            self.bat_ctrl_reactivate.succeed()  # "reactivate"
            self.bat_ctrl_reactivate = env.event()
            yield env.timeout(randint(60, 360))
            print('Stop parking at', env.now)
    def bat_ctrl(self, env):
        while True:
            print('Bat. ctrl. passivating at', env.now)
            yield self.bat_ctrl_reactivate  # "passivate"
            print('Bat. ctrl. reactivated at', env.now)
            # Intelligent charging behavior here …
            yield env.timeout(randint(30, 90))
env = simpy.Environment()
ev = EV(env)
env.run(until=150)

Bat. ctrl. passivating at 0
Start parking at 29
Bat. ctrl. reactivated at 29
Bat. ctrl. passivating at 60
Stop parking at 131


**Waiting for another process to terminate**

In [0]:
import simpy
class EV:
    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive(env))
    def drive(self, env):
        while True:
            # Drive for 20-40 min
            yield env.timeout(randint(20, 40))
            # Park for 1–6 hours
            print('Start parking at', env.now)
            charging = env.process(self.bat_ctrl(env))
            parking = env.timeout(randint(60, 360))
            yield charging & parking
            print('Stop parking at', env.now)
    def bat_ctrl(self, env):
        print('Bat. ctrl. started at', env.now)
        # Intelligent charging behavior here …
        yield env.timeout(randint(30, 90))
        print('Bat. ctrl. done at', env.now)

env = simpy.Environment()
ev = EV(env)
env.run(until=310)

Start parking at 29
Bat. ctrl. started at 29
Bat. ctrl. done at 83
Stop parking at 305


**Interrupting another process**

In [0]:
class EV:
     def __init__(self, env):
         self.env = env
         self.drive_proc = env.process(self.drive(env))

     def drive(self, env):
         while True:
             # Drive for 20-40 min
             yield env.timeout(randint(20, 40))

             # Park for 1 hour
             print('Start parking at', env.now)
             charging = env.process(self.bat_ctrl(env))
             parking = env.timeout(60)
             yield charging | parking
             if not charging.triggered:
                 # Interrupt charging if not already done.
                 charging.interrupt('Need to go!')
             print('Stop parking at', env.now)

     def bat_ctrl(self, env):
         print('Bat. ctrl. started at', env.now)
         try:
             yield env.timeout(randint(60, 90))
             print('Bat. ctrl. done at', env.now)
         except simpy.Interrupt as i:
             # Onoes! Got interrupted before the charging was done.
             print('Bat. ctrl. interrupted at', env.now, 'msg:',
                   i.cause)

env = simpy.Environment()
ev = EV(env)
env.run(until=100)

Start parking at 31
Bat. ctrl. started at 31
Stop parking at 91
Bat. ctrl. interrupted at 91 msg: Need to go!


# **7.	WAP to Interrupting another process in case of car charging simulation.**

In [0]:
class EV:
     def __init__(self, env):
         self.env = env
         self.drive_proc = env.process(self.drive(env))

     def drive(self, env):
         while True:
             # Drive for 20-40 min
             yield env.timeout(randint(20, 40))

             # Park for 1 hour
             print('Start parking at', env.now)
             charging = env.process(self.bat_ctrl(env))
             parking = env.timeout(60)
             yield charging | parking
             if not charging.triggered:
                 # Interrupt charging if not already done.
                 charging.interrupt('Need to go!')
             print('Stop parking at', env.now)

     def bat_ctrl(self, env):
         print('Bat. ctrl. started at', env.now)
         try:
             yield env.timeout(randint(60, 90))
             print('Bat. ctrl. done at', env.now)
         except simpy.Interrupt as i:
             # Onoes! Got interrupted before the charging was done.
             print('Bat. ctrl. interrupted at', env.now, 'msg:',
                   i.cause)

env = simpy.Environment()
ev = EV(env)
env.run(until=100)
#It is a methot of Process Interaction

Start parking at 28
Bat. ctrl. started at 28
Stop parking at 88
Bat. ctrl. interrupted at 88 msg: Need to go!


# **8.	What is the different between dependent and independent processes?**

In [0]:
An independent rescources is the rescource that is changed or controlled in a scientific experiment to test the effects on the dependent rescource.
A dependent Rescource is the Rescource being tested and measured in a scientific experiment